In [1]:
!pip install dotenv

In [2]:
!pip install langchain

In [3]:
!pip install langchain_community
!pip install langchain_groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 4.5 MB/s eta 0:00:00


In [9]:
#import os
import json
import datetime
#from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import load_summarize_chain
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.documents import Document

# Load environment variables from .env
#load_dotenv(".env")
#groq_api_key = os.getenv("GROQ_API_KEY")
#serper_api_key = os.getenv("SERPER_API_KEY")

groq_api_key="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"
serper_api_key="03ea0164240c44d2c8c64fe2b0f0f9d74b21cdfd"


# Initialize Groq LLaMA model and summarization tools
llm = ChatGroq(model="llama3-8b-8192", api_key=groq_api_key)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
summarize_chain = load_summarize_chain(llm, chain_type="map_reduce")

# Department info
DEPARTMENT_CONTACTS = {
    "Electricity Board": {"phone": "1800-112-233", "email": "power@civic.gov.in"},
    "Department of Water Resources": {"phone": "1800-221-445", "email": "water@civic.gov.in"},
    "Road Development": {"phone": "1800-443-556", "email": "roads@civic.gov.in"},
    "Health Ministry": {"phone": "1800-777-999", "email": "health@civic.gov.in"},
    "Sanitation": {"phone": "1800-333-122", "email": "cleanliness@civic.gov.in"}
}
DEPARTMENTS = list(DEPARTMENT_CONTACTS.keys())

def classify_departments(text):
    prompt = f"""Classify this complaint into one or more of the following departments:
{', '.join(DEPARTMENTS)}.
Complaint: {text}
Departments:"""
    response = llm.invoke(prompt)
    return [dept.strip() for dept in response.content.split(",") if dept.strip() in DEPARTMENTS]

def get_severity_score(text):
    prompt = f"Rate the severity of this complaint from 1 (least severe) to 5 (most severe):\n{text}\nSeverity:"
    try:
        response = llm.invoke(prompt)
        score = int("".join(filter(str.isdigit, response)))
        return min(max(score, 1), 5)
    except Exception:
        return 3

def get_contact_info(departments):
    return {dept: DEPARTMENT_CONTACTS[dept] for dept in departments if dept in DEPARTMENT_CONTACTS}

def summarize_text(text):
    texts = text_splitter.split_text(text)
    docs = [Document(page_content=t) for t in texts]
    return summarize_chain.run(docs)

def fetch_interim_suggestions(complaint_text, department):
    search = GoogleSerperAPIWrapper(api_key=serper_api_key)
    query = f"What should a citizen do while waiting for {department} to resolve: {complaint_text}"
    result = search.run(query)
    return summarize_text(result).split("\n")

def generate_officer_brief(summary, severity, departments):
    dept_str = ", ".join(departments)
    return f"A complaint has been received regarding {summary}. The issue is rated {severity}/5 in severity and is forwarded to the {dept_str}."

def process_complaint(text):
    departments = classify_departments(text)
    severity = get_severity_score(text)
    contact_info = get_contact_info(departments)
    summary = summarize_text(text)
    suggestions = []
    for dept in departments:
        suggestions.extend(fetch_interim_suggestions(text, dept))
    officer_brief = generate_officer_brief(summary, severity, departments)

    structured = {
        "original_text": text,
        "departments": departments,
        "severity": severity,
        "suggestions": suggestions,
        "contact_info": contact_info,
        "timestamp": datetime.datetime.utcnow().isoformat(),
        "status": "pending",
        "officer_brief": officer_brief
    }

    with open("structured_complaints.json", "a") as f:
        json.dump(structured, f, indent=2)
        f.write(",\n")

    return structured

if __name__ == "__main__":
    user_input = input("Enter your civic complaint: ").strip()
    if user_input:
        result = process_complaint(user_input)
        print(json.dumps(result, indent=2))
    else:
        print("Complaint text cannot be empty.")


Enter your civic complaint: last friday a kid died due to thirstIn our locality, the water supply is extremely erratic, and there are often entire days when no water is supplied at all. This has been causing significant distress to residents, especially families with children. Kindly address this issue and ensure a regular water supply to avoid further inconvenience.
{
  "original_text": "last friday a kid died due to thirstIn our locality, the water supply is extremely erratic, and there are often entire days when no water is supplied at all. This has been causing significant distress to residents, especially families with children. Kindly address this issue and ensure a regular water supply to avoid further inconvenience.",
  "departments": [],
  "severity": 3,
  "suggestions": [],
  "contact_info": {},
  "timestamp": "2025-04-12T16:44:04.673811",
  "status": "pending",
  "officer_brief": "A complaint has been received regarding A child died due to dehydration caused by a lack of wat

In [ ]:
import os
import json
import datetime
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import load_summarize_chain
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.documents import Document

# Load environment variables
groq_api_key="#####"
serper_api_key="###"

# Initialize LLM and tools
llm = ChatGroq(model="llama3-8b-8192", api_key=groq_api_key)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
summarize_chain = load_summarize_chain(llm, chain_type="map_reduce")
search = GoogleSerperAPIWrapper(api_key=serper_api_key)

# Departments and contacts
DEPARTMENT_CONTACTS = {
    "Electricity Board": {"phone": "1800-112-233", "email": "power@civic.gov.in"},
    "Department of Water Resources": {"phone": "1800-221-445", "email": "water@civic.gov.in"},
    "Road Development": {"phone": "1800-443-556", "email": "roads@civic.gov.in"},
    "Health Ministry": {"phone": "1800-777-999", "email": "health@civic.gov.in"},
    "Sanitation": {"phone": "1800-333-122", "email": "cleanliness@civic.gov.in"}
}
DEPARTMENTS = list(DEPARTMENT_CONTACTS.keys())

# Classify departments using LLaMA
def classify_departments(text):
    prompt = f"""Given this complaint:
{text}
Classify it into one or more of the following departments:
{', '.join(DEPARTMENTS)}.
Return only department names as a comma-separated list."""
    response = llm.invoke(prompt)
    return [d.strip() for d in response.content.split(",") if d.strip() in DEPARTMENTS]

# Assess severity using LLaMA
def get_severity_score(text):
    prompt = f"""Assess the severity of this civic complaint on a scale of 1 (least) to 5 (most severe).
Complaint:
{text}
Severity (number only):"""
    try:
        response = llm.invoke(prompt).content
        score = int("".join(filter(str.isdigit, response)))
        return min(max(score, 1), 5)
    except Exception:
        return 3

# Summarize using LLaMA
def summarize_text(text):
    docs = [Document(page_content=t) for t in text_splitter.split_text(text)]
    return summarize_chain.run(docs)

# Get contact info
def get_contact_info(departments):
    return {d: DEPARTMENT_CONTACTS[d] for d in departments if d in DEPARTMENT_CONTACTS}

# Fetch suggestions using Serper and LLaMA
def fetch_interim_suggestions(complaint_text, department):
    query = f"What should a citizen do while waiting for {department} to resolve: {complaint_text}"
    result = search.run(query)
    summary = summarize_text(result)
    return [line.strip("-• ") for line in summary.split("\n") if line.strip()]

# Generate officer brief
def generate_officer_brief(summary, severity, departments):
    dept_str = ", ".join(departments) if departments else "relevant authority"
    return f"A complaint has been received regarding {summary}. The issue is rated {severity}/5 in severity and is forwarded to the {dept_str}."

# Process complaint
def process_complaint(text):
    departments = classify_departments(text)
    severity = get_severity_score(text)
    summary = summarize_text(text)
    contact_info = get_contact_info(departments)
    suggestions = []
    for dept in departments:
        suggestions += fetch_interim_suggestions(text, dept)
    officer_brief = generate_officer_brief(summary, severity, departments)

    structured = {
        "original_text": text,
        "departments": departments,
        "severity": severity,
        "suggestions": suggestions,
        "contact_info": contact_info,
        "timestamp": datetime.datetime.utcnow().isoformat(),
        "status": "pending",
        "officer_brief": officer_brief
    }

    with open("structured_complaints.json", "a") as f:
        json.dump(structured, f, indent=2)
        f.write(",\n")

    return structured

# CLI input
if __name__ == "__main__":
    user_input = input("Enter your civic complaint: ").strip()
    if user_input:
        result = process_complaint(user_input)
        print(json.dumps(result, indent=2))
    else:
        print("Complaint text cannot be empty.")


Enter your civic complaint: .i was out out of station when i came back my whole family died of dehydration in our locality, the water supply is extremely erratic, and there are often entire days when no water is supplied at all. This has been causing significant distress to residents, especially families with children. Kindly address this issue and ensure a regular water supply to avoid further inconvenience.
{
  "original_text": ".i was out out of station when i came back my whole family died of dehydration in our locality, the water supply is extremely erratic, and there are often entire days when no water is supplied at all. This has been causing significant distress to residents, especially families with children. Kindly address this issue and ensure a regular water supply to avoid further inconvenience.",
  "departments": [
    "Department of Water Resources",
    "Sanitation"
  ],
  "severity": 5,
  "suggestions": [
    "Here is a concise summary:",
    "The text appears to be a 

In [13]:
import os
import json
import datetime
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import load_summarize_chain
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.documents import Document

# Load environment variables
groq_api_key="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"
serper_api_key="03ea0164240c44d2c8c64fe2b0f0f9d74b21cdfd"

# Initialize LLM and tools
llm = ChatGroq(model="llama3-8b-8192", api_key=groq_api_key)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
summarize_chain = load_summarize_chain(llm, chain_type="map_reduce")
search = GoogleSerperAPIWrapper(api_key=serper_api_key)

# Departments and contacts
DEPARTMENT_CONTACTS = {
    "Electricity Board": {"phone": "1800-112-233", "email": "power@civic.gov.in"},
    "Department of Water Resources": {"phone": "1800-221-445", "email": "water@civic.gov.in"},
    "Road Development": {"phone": "1800-443-556", "email": "roads@civic.gov.in"},
    "Health Ministry": {"phone": "1800-777-999", "email": "health@civic.gov.in"},
    "Sanitation": {"phone": "1800-333-122", "email": "cleanliness@civic.gov.in"}
}
DEPARTMENTS = list(DEPARTMENT_CONTACTS.keys())

# Classify departments using LLaMA
def classify_departments(text):
    prompt = f"""Given this complaint:
{text}
Classify it into one or more of the following departments:
{', '.join(DEPARTMENTS)}.
Return only department names as a comma-separated list."""
    response = llm.invoke(prompt)
    return [d.strip() for d in response.content.split(",") if d.strip() in DEPARTMENTS]

# Assess severity using LLaMA
def get_severity_score(text):
    prompt = f"""Assess the severity of this civic complaint on a scale of 1 (least) to 5 (most severe).
Complaint:
{text}
Severity (number only):"""
    try:
        response = llm.invoke(prompt).content
        score = int("".join(filter(str.isdigit, response)))
        return min(max(score, 1), 5)
    except Exception:
        return 3

# Summarize using LLaMA
def summarize_text(text):
    docs = [Document(page_content=t) for t in text_splitter.split_text(text)]
    return summarize_chain.run(docs)

# Get contact info
def get_contact_info(departments):
    return {d: DEPARTMENT_CONTACTS[d] for d in departments if d in DEPARTMENT_CONTACTS}

# Fetch suggestions using Serper and LLaMA
def fetch_interim_suggestions(complaint_text, department):
    query = f"What should a citizen do while waiting for {department} to resolve: {complaint_text}"
    result = search.run(query)
    summary = summarize_text(result)
    return [line.strip("-• ") for line in summary.split("\n") if line.strip()]

# Generate officer brief
def generate_officer_brief(summary, severity, departments):
    dept_str = ", ".join(departments) if departments else "relevant authority"
    return f"A complaint has been received regarding {summary}. The issue is rated {severity}/5 in severity and is forwarded to the {dept_str}."

# Process complaint
def process_complaint(text):
    departments = classify_departments(text)
    severity = get_severity_score(text)
    summary = summarize_text(text)
    contact_info = get_contact_info(departments)
    suggestions = []
    for dept in departments:
        suggestions.extend(fetch_interim_suggestions(text, dept))
    officer_brief = generate_officer_brief(summary, severity, departments)

    # Format output as simple text
    output = f"Original Complaint: {text}\n"
    output += f"Departments: {', '.join(departments)}\n"
    output += f"Severity: {severity}/5\n"
    output += "Suggestions:\n"
    for suggestion in suggestions:
        output += f"- {suggestion}\n"
    output += "Contact Info:\n"
    for dept, info in contact_info.items():
        output += f"{dept}: Phone - {info['phone']}, Email - {info['email']}\n"
    output += f"Timestamp: {datetime.datetime.utcnow().isoformat()}\n"
    output += "Status: pending\n"
    output += f"Officer Brief: {officer_brief}\n"

    # Save output to file
    with open("structured_complaints.txt", "a") as f:
        f.write(output)
        f.write("\n---\n")  # Separator between complaints

    return output

# CLI input
if __name__ == "__main__":
    user_input = input("Enter your civic complaint: ").strip()
    if user_input:
        result = process_complaint(user_input)
        print(result)  # Print the formatted text output
    else:
        print("Complaint text cannot be empty.")


Enter your civic complaint: .i was out out of station when i came back my whole family died of dehydration in our locality, the water supply is extremely erratic, and there are often entire days when no water is supplied at all. This has been causing significant distress to residents, especially families with children. Kindly address this issue and ensure a regular water supply to avoid further inconvenience.
Original Complaint: .i was out out of station when i came back my whole family died of dehydration in our locality, the water supply is extremely erratic, and there are often entire days when no water is supplied at all. This has been causing significant distress to residents, especially families with children. Kindly address this issue and ensure a regular water supply to avoid further inconvenience.
Departments: Department of Water Resources, Health Ministry
Severity: 5/5
Suggestions:
- There is no Jerry and his team, nor is there a project mentioned in the original text. The or